# Libraries

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import re
import string

# Load Dataset

In [3]:
df=pd.read_csv('recipes.csv')
df

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
0,38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,NaN,"c(""Toss 2 cups berries with sugar."", ""Let stan..."
1,39,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29T13:12:00Z,Make and share this Biryani recipe from Food.com.,"c(""https://img.sndimg.com/food/image/upload/w_...",...,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,NaN,"c(""Soak saffron in warm milk for 5 minutes and..."
2,40,Best Lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,1999-09-05T19:52:00Z,This is from one of my first Good House Keepi...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,NaN,"c(""Into a 1 quart Jar with tight fitting lid, ..."
3,41,Carina's Tofu-Vegetable Kebabs,1586,Cyclopz,PT20M,PT24H,PT24H20M,1999-09-03T14:54:00Z,This dish is best prepared a day in advance to...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,2.0,4 kebabs,"c(""Drain the tofu, carefully squeezing out exc..."
4,42,Cabbage Soup,1538,Duckie067,PT30M,PT20M,PT50M,1999-09-19T06:19:00Z,Make and share this Cabbage Soup recipe from F...,"""https://img.sndimg.com/food/image/upload/w_55...",...,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0,NaN,"c(""Mix everything together and bring to a boil..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522512,541379,Meg's Fresh Ginger Gingerbread,2002090414,rdsxc,PT35M,PT1H,PT1H35M,2020-12-22T15:27:00Z,Make and share this Meg's Fresh Ginger Gingerb...,character(0),...,7.6,54.4,278.2,48.5,0.8,22.8,3.9,8.0,1 8x8 cake pan,"c(""Preheat oven to 350&deg;F Grease an 8x8 cak..."
522513,541380,Roast Prime Rib au Poivre with Mixed Peppercorns,211566,Denver cooks,PT3H,PT30M,PT3H30M,2020-12-22T15:32:00Z,"White, black, green, and pink peppercorns add ...","""https://img.sndimg.com/food/image/upload/w_55...",...,71.4,433.8,766.3,3.2,0.7,0.1,117.0,8.0,1 Roast,"c(""Position rack in center of oven and preheat..."
522514,541381,Kirshwasser Ice Cream,2001131545,Jonathan F.,PT3H,PT1H,PT4H,2020-12-22T15:33:00Z,Make and share this Kirshwasser Ice Cream reci...,character(0),...,72.6,470.9,192.5,33.9,0.0,17.3,12.8,6.0,NaN,"c(""heat half and half and heavy cream to a sim..."
522515,541382,Quick & Easy Asian Cucumber Salmon Rolls,2001004241,CLUBFOODY,NaN,PT15M,PT15M,2020-12-22T22:11:00Z,"Extremely quick and easy to make, these are gr...","""https://img.sndimg.com/food/image/upload/w_55...",...,0.1,2.9,100.5,0.3,0.0,0.1,2.4,NaN,20 rolls,"c(""In a small bowl, combine mayo and wasabi pa..."


In [4]:
df.columns

Index(['RecipeId', 'Name', 'AuthorId', 'AuthorName', 'CookTime', 'PrepTime',
       'TotalTime', 'DatePublished', 'Description', 'Images', 'RecipeCategory',
       'Keywords', 'RecipeIngredientQuantities', 'RecipeIngredientParts',
       'AggregatedRating', 'ReviewCount', 'Calories', 'FatContent',
       'SaturatedFatContent', 'CholesterolContent', 'SodiumContent',
       'CarbohydrateContent', 'FiberContent', 'SugarContent', 'ProteinContent',
       'RecipeServings', 'RecipeYield', 'RecipeInstructions'],
      dtype='object')

Select required columns

In [6]:
df=df[['Name','Calories', 'FatContent', 'SaturatedFatContent','CholesterolContent', 'SodiumContent', 'CarbohydrateContent','FiberContent', 'SugarContent', 'ProteinContent','RecipeInstructions']]

In [7]:
df.shape

(522517, 11)

In [8]:
# rows: 522517   ,   columns: 11

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522517 entries, 0 to 522516
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Name                 522517 non-null  object 
 1   Calories             522517 non-null  float64
 2   FatContent           522517 non-null  float64
 3   SaturatedFatContent  522517 non-null  float64
 4   CholesterolContent   522517 non-null  float64
 5   SodiumContent        522517 non-null  float64
 6   CarbohydrateContent  522517 non-null  float64
 7   FiberContent         522517 non-null  float64
 8   SugarContent         522517 non-null  float64
 9   ProteinContent       522517 non-null  float64
 10  RecipeInstructions   522517 non-null  object 
dtypes: float64(9), object(2)
memory usage: 43.9+ MB


In [10]:
# No Null values

# Save Dataset

In [11]:
df.to_csv('food.csv', index=False)

# Data Preprocessing

In [12]:
# Creating function for clean column RecipeInstructions
def clean_instructions(text):
    text = text.lstrip('c("').rstrip('",)') # Remove the 'c' at the start and quotes
    text = text.translate(str.maketrans("", "", string.punctuation))# Remove any remaining special characters if needed
    return text

# Appling function
df['RecipeInstructions'] = df['RecipeInstructions'].apply(clean_instructions)

C:\Users\Pc\AppData\Local\Temp\ipykernel_14452\393621297.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['RecipeInstructions'] = df['RecipeInstructions'].apply(clean_instructions)


# Functions for Calculations

Calculate BMR using Mifflin-St Jeor Formula

In [13]:
def calculate_bmr(weight, height, age, sex):
    if sex == "male":
        return (9.99 * weight) + (6.25 * height) - (4.92 * age) + 5
    else: # female
        return (9.99 * weight) + (6.25 * height) - (4.92 * age) - 161

Calculate Total Daily Energy Expenditure based on activity levels

In [14]:
def calculate_tdee(bmr, activity_level):
    activity_lvls = {
        "sedentary": 1.2,
        "lightly_active": 1.375,
        "moderate": 1.55,
        "very_active": 1.725,
        "extra_active": 1.9
    }
    return bmr * activity_lvls.get(activity_level, 1.2) # tdee


Calculate Calories Based on Goal

In [15]:
def adjust_calories(tdee, goal):
    if goal == "weight_loss":
        return tdee - 500
    elif goal == "weight_gain":
        return tdee + 500
    else:  # Maintenance
        return tdee

Macronutrient Distribution Ratios

In [16]:
def calculate_macros(calories, goal):
    if goal == "weight_loss":
        protein_ratio, fat_ratio, carb_ratio = 0.3, 0.3, 0.4
    elif goal == "weight_gain":
        protein_ratio, fat_ratio, carb_ratio = 0.25, 0.3, 0.45
    else:  #  maintenance
        protein_ratio, fat_ratio, carb_ratio = 0.3, 0.3, 0.4
    
    protein = (calories * protein_ratio) / 4
    fat = (calories * fat_ratio) / 9
    carbs = (calories * carb_ratio) / 4
    return protein, fat, carbs

# User inputs

In [17]:
age=int(input('Age: '))
sex=input('Gender: ')
weight=int(input('Weight: '))
height=int(input('Height: '))
activity_level=input('Activity Level(sedentary,lightly_active,moderate,very_active,extra_active): ')
goal=input('Goal(weight_loss,weight_gain,maintenance): ')
diet_preference=input('diet preference(vegetarian,non-vegetarian): ')
allergies=input('Allergies: ')
n=int(input('Number of recipies: '))

In [18]:
user_profile = {}
user_profile["age"] =age
user_profile["sex"] =sex
user_profile["weight"] =weight
user_profile["height"] =height
user_profile["activity_level"] =activity_level
user_profile["goal"] =goal
user_profile["diet_preference"] =diet_preference
user_profile["allergies"] =[allergies]

# Compute User Nutritional Needs

In [19]:
bmr = calculate_bmr(user_profile["weight"], user_profile["height"], user_profile["age"], user_profile["sex"])
tdee = calculate_tdee(bmr, user_profile["activity_level"])
target_calories = adjust_calories(tdee, user_profile["goal"])
target_protein, target_fat, target_carbs = calculate_macros(target_calories, user_profile["goal"])

# Filter Recipes Based on Diet Preferences

In [20]:
filtered_diet = df.copy()
if user_profile["diet_preference"] in ["vegetarian", "vegan"]:
    filtered_diet = filtered_diet[filtered_diet["RecipeInstructions"].str.contains(user_profile["diet_preference"], case=False, na=False)]

# for allergen in user_profile["allergies"]:
#     filtered_diet = filtered_diet[~filtered_diet["RecipeInstructions"].str.contains(allergen, case=False, na=False)]

# Content-Based Recommendation Using Cosine Similarity

In [21]:
def recommend_meals(user_calories, user_fat, user_carbs, user_protein, top_n):
    # User profile vector includes all 9 features, and we fill missing features with zeros (or use other defaults)
    user_profile_vector = np.array([[user_calories, user_fat, user_carbs, user_protein, 0, 0, 0, 0, 0]])
    
    # Compute Cosine Similarity
    similarities = cosine_similarity(user_profile_vector, filtered_diet.iloc[:, 1:10].values)  # Adjusting for 9 features
    
    # Rank meals based on similarity
    filtered_diet["Similarity"] = similarities[0]
    top_recommendations = filtered_diet.sort_values(by="Similarity", ascending=False).head(top_n)
    
    # Return recommendations in the desired format
    recommendations = []
    for _, row in top_recommendations.iterrows():
        recommendation = {
            "Name": row["Name"],
            "Calories": row["Calories"],
            "ProteinContent": row["ProteinContent"],
            "FatContent": row["FatContent"],
            "CarbohydrateContent": row["CarbohydrateContent"],
            "SodiumContent": row["SodiumContent"],
            "RecipeInstructions": row["RecipeInstructions"]
        }
        recommendations.append(recommendation)
    
    return recommendations

# Get Recommendations

In [22]:
recommendations = recommend_meals(target_calories, target_fat, target_carbs, target_protein,n)

In [23]:
for rec in recommendations:
    print(f"Name: {rec['Name']}")
    print(f"Calories: {rec['Calories']}")
    print(f"ProteinContent: {rec['ProteinContent']}")
    print(f"FatContent: {rec['FatContent']}")
    print(f"CarbohydrateContent: {rec['CarbohydrateContent']}")
    print(f"SodiumContent: {rec['SodiumContent']}")
    print(f"RecipeInstructions: {rec['RecipeInstructions']}")
    print("\n" + "-"*50 + "\n")

Name: Sauteed Greens
Calories: 61.9
ProteinContent: 0.1
FatContent: 6.8
CarbohydrateContent: 0.5
SodiumContent: 0.4
RecipeInstructions: Pour 12 inch of water into a large pot and bring to a boil Add greens cover and steam until wilted about a minute Drain and chop In a large skillet heat the oil and add garlic Sauté over medium heat 30 seconds Sauté greens approximately five minutes Season with salt and tamari soy sauce Serve immediately Variations Instead of sautéing with oil and garlic steam the greens and flavour with a 12 teaspoon of fresh ginger juice lemon juice or umeboshi vinegar Ginger juice can be made by grating ginger and squeezing the juice out of the pulp Garnish with carrot slivers sesame seeds and enoki mushrooms 
Source Vegetarian Tastes of Toronto p 28

--------------------------------------------------

Name: The Best Vegetarian French Onion Soup
Calories: 197.3
ProteinContent: 0.8
FatContent: 6.8
CarbohydrateContent: 11.3
SodiumContent: 7.2
RecipeInstructions: Heat 